In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# let's check that the initial data from murilo is theres
! gcloud storage ls gs://dsgt-clef-birdclef-2024/data/raw/birdclef-2023 | head

gs://dsgt-clef-birdclef-2024/data/raw/birdclef-2023/eBird_Taxonomy_v2021.csv
gs://dsgt-clef-birdclef-2024/data/raw/birdclef-2023/sample_submission.csv
gs://dsgt-clef-birdclef-2024/data/raw/birdclef-2023/train_metadata.csv
gs://dsgt-clef-birdclef-2024/data/raw/birdclef-2023/test_soundscapes/
gs://dsgt-clef-birdclef-2024/data/raw/birdclef-2023/train_audio/


In [3]:
from birdclef.utils import get_spark
from IPython.display import Image, display

spark = get_spark()
display(spark)

# read straight from the bucket
df_meta = spark.read.csv(
    "gs://dsgt-clef-birdclef-2024/data/raw/birdclef-2023/train_metadata.csv"
)
df_meta.printSchema()
df_meta.show(vertical=True, n=1, truncate=100)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/19 06:13:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)

-RECORD 0----------------
 _c0  | primary_label    
 _c1  | secondary_labels 
 _c2  | type             
 _c3  | latitude         
 _c4  | longitude        
 _c5  | scientific_name  
 _c6  | common_name      
 _c7  | author           
 _c8  | license          
 _c9  | rating           
 _c10 | url              
 _c11 | filename         
only showing top 1 row



In [4]:
df_meta.show(n=5)

+-------------+----------------+--------+--------+---------+------------------+--------------------+-------------+--------------------+------+--------------------+--------------------+
|          _c0|             _c1|     _c2|     _c3|      _c4|               _c5|                 _c6|          _c7|                 _c8|   _c9|                _c10|                _c11|
+-------------+----------------+--------+--------+---------+------------------+--------------------+-------------+--------------------+------+--------------------+--------------------+
|primary_label|secondary_labels|    type|latitude|longitude|   scientific_name|         common_name|       author|             license|rating|                 url|            filename|
|      abethr1|              []|['song']|  4.3906|  38.2788|Turdus tephronotus|African Bare-eyed...|Rolf A. de By|Creative Commons ...|   4.0|https://www.xeno-...|abethr1/XC128013.ogg|
|      abethr1|              []|['call']| -2.9524|  38.2921|Turdus tephrono

In [5]:
from matplotlib import pyplot as plt
import pandas as pd

# Group by species and count
species_counts = df_meta.groupBy("_c5").count().orderBy("count", ascending=False)

# Convert to Pandas DataFrame for plotting
species_df = species_counts.toPandas()

# Get the top 5 species
top_5 = species_df.head(5)

/tmp/ipykernel_26857/1609202181.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/home/acheung/birdclef-2024/venv/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:111: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  PyArrow >= 4.0.0 must be installed; however, it was not found.
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warn(msg)


In [6]:
top_5

,_c5,count
0,Actitis hypoleucos,500
1,Hirundo rustica,499
2,Phylloscopus trochilus,499
3,Luscinia luscinia,498
4,Motacilla flava,498


In [7]:
from pyspark.sql import Window
from pyspark.sql.functions import desc, row_number

# filter the top num_species species
num_species = 3
top_species = species_df.head(num_species)["_c5"].values.tolist()
df_meta_top = df_meta.filter(df_meta._c5.isin(top_species))

# take first 10 of every species
window = Window.partitionBy("_c5").orderBy(desc("_c9"))
df_meta_top = df_meta_top.withColumn("row_num", row_number().over(window))
df_meta_top = df_meta_top.filter(df_meta_top.row_num <= 10)

In [8]:
df_meta_top.count()

30

In [9]:
df_meta_top.show()

+------+---+--------------------+--------+--------+------------------+----------------+----------------+--------------------+---+--------------------+-------------------+-------+
|   _c0|_c1|                 _c2|     _c3|     _c4|               _c5|             _c6|             _c7|                 _c8|_c9|                _c10|               _c11|row_num|
+------+---+--------------------+--------+--------+------------------+----------------+----------------+--------------------+---+--------------------+-------------------+-------+
|comsan| []|['female', 'fligh...| 52.3527| 20.9197|Actitis hypoleucos|Common Sandpiper|  Jarek Matusiak|Creative Commons ...|5.0|https://www.xeno-...|comsan/XC129378.ogg|      1|
|comsan| []|            ['call']| 54.2697|  8.8683|Actitis hypoleucos|Common Sandpiper|   Volker Arnold|Creative Commons ...|5.0|https://www.xeno-...|comsan/XC142817.ogg|      2|
|comsan| []|            ['call']| 55.0194| 82.8918|Actitis hypoleucos|Common Sandpiper|      Sander Bot|C

## Encodec with a single recording

In [49]:
# load top recording with gs://dsgt-clef-birdclef-2024/data/raw/birdclef-2023/_c11
# df_meta_top = df_meta_top.withColumn(
#     "recording_path",
#     f"gs://dsgt-clef-birdclef-2024/data/raw/birdclef-2023/{df_meta_top['_c11']}",
# )
# get first row of df_meta_top
# row = df_meta_top.head()

# read from google cloud storage


# df_single = spark.read(f"gs://dsgt-clef-birdclef-2024/data/raw/birdclef-2023/train_audio/{row._c11}")
# single_recording = df_single.toPandas()
# single_recording

/home/acheung/birdclef-2024/venv/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:111: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  PyArrow >= 4.0.0 must be installed; however, it was not found.
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warn(msg)


,_c0,_c1
0,OggS         B��v    �� vorbis     }      ...,(���0��5(���0�ԃ B���I5��gAx�iA!�$AH��A�...
1,�,None
2, �  @Q�q ɑ ɱ ,None
3,Y     �H��H��H�$Y�%Y�%Y�扪,˲
4, H  PQ Eq,None
...,...,...
581,�d�ybP��?y��O�l E�\tZ�V��Ҹ��,None
582,"4j� �""��   �=D��|""��r���4��I�*�O8��g3;",None
583,y�/�=BW�l�&�a8pΓusL] ������ ��[k8g�dk09c��,����&��3���f ����'; 6;Z7�V��AV_���l��۔��]��b�...
584,"��m��ߖl~��2[�C~��^ ""Fp=��<ϳ c$�y�Hz�S1J��...",-/����$��t��e�vXk��U


In [19]:
row = df_meta_top.head()
single_path = f"data/raw/birdclef-2023/train_audio/{row._c11}"

In [26]:
from google.cloud import storage
import io

client = storage.Client()
bucket = client.get_bucket('dsgt-clef-birdclef-2024')
blob = bucket.blob(single_path)
file_stream = io.BytesIO()
blob.download_to_file(file_stream)
file_stream.seek(0)

0

In [24]:
from encodec import EncodecModel
from encodec.utils import convert_audio

import torchaudio
import torch

# Instantiate a pretrained EnCodec model
model = EncodecModel.encodec_model_24khz()
# The number of codebooks used will be determined bythe bandwidth selected.
# E.g. for a bandwidth of 6kbps, `n_q = 8` codebooks are used.
# Supported bandwidths are 1.5kbps (n_q = 2), 3 kbps (n_q = 4), 6 kbps (n_q = 8) and 12 kbps (n_q =16) and 24kbps (n_q=32).
# For the 48 kHz model, only 3, 6, 12, and 24 kbps are supported. The number
# of codebooks for each is half that of the 24 kHz model as the frame rate is twice as much.
model.set_target_bandwidth(6.0)

/home/acheung/birdclef-2024/venv/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [27]:
# Load and pre-process the audio waveform
wav, sr = torchaudio.load(file_stream)
wav = convert_audio(wav, sr, model.sample_rate, model.channels)
wav = wav.unsqueeze(0)

# Extract discrete codes from EnCodec
with torch.no_grad():
    encoded_frames = model.encode(wav)
codes = torch.cat([encoded[0] for encoded in encoded_frames], dim=-1)  # [B, n_q, T]

In [28]:
codes.shape

torch.Size([1, 8, 854])

In [30]:
# decode
with torch.no_grad():
    decoded_frames = model.decode(encoded_frames)
# save
torchaudio.save("data/reconstructed.wav", decoded_frames, model.sample_rate)

RuntimeError: Input Tensor has to be 2D.
Exception raised from validate_audio_input at /__w/audio/audio/pytorch/audio/src/libtorio/ffmpeg/stream_writer/tensor_converter.cpp:31 (most recent call first):
frame #0: c10::Error::Error(c10::SourceLocation, std::string) + 0x57 (0x7fc1b424ad87 in /home/acheung/birdclef-2024/venv/lib/python3.10/site-packages/torch/lib/libc10.so)
frame #1: c10::detail::torchCheckFail(char const*, char const*, unsigned int, char const*) + 0x68 (0x7fc1b41fb828 in /home/acheung/birdclef-2024/venv/lib/python3.10/site-packages/torch/lib/libc10.so)
frame #2: <unknown function> + 0x579f7 (0x7fc1b40aa9f7 in /home/acheung/birdclef-2024/venv/lib/python3.10/site-packages/torio/lib/libtorio_ffmpeg4.so)
frame #3: <unknown function> + 0x5826c (0x7fc1b40ab26c in /home/acheung/birdclef-2024/venv/lib/python3.10/site-packages/torio/lib/libtorio_ffmpeg4.so)
frame #4: torio::io::TensorConverter::convert(at::Tensor const&) + 0x33 (0x7fc1b40ace93 in /home/acheung/birdclef-2024/venv/l